In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

/home/vicky/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/vicky/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/vicky/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/vicky/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
data = pd.read_csv("../Large_data/feature/bnp-paribas-card-claim/train.csv", nrows=20000)  
data.shape

(20000, 133)

In [4]:
#feature selection should be done after data preprocessing
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape
#[col for col in data.columns if data[col].isnull().sum() > 0] #presence of null value

(20000, 114)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=["target","ID"], axis =1),
    data["target"],
    test_size = 0.3,
    random_state = 0
)
del data
X_train.shape, X_test.shape

((14000, 112), (6000, 112))

In [6]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j] > threshold):
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print("Correlated Features: ", len(set(corr_features)))

Correlated Features:  47


In [7]:
X_train.drop(labels=corr_features, axis =1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((14000, 65), (6000, 65))

In [11]:
efs1 = EFS(RandomForestClassifier(n_jobs=4, random_state=0),
          min_features=1,
          max_features=4,
          print_progress=True,
          scoring='roc_auc',
          cv=3)

efs1 = efs1.fit(np.array(X_train[X_train.columns[0:4]].fillna(0)), y_train)

Features: 15/15

In [14]:
selected_feat = X_train.columns[list(efs1.best_idx_)]
selected_feat

Index(['v2', 'v4'], dtype='object')

In [15]:
def run_randomForest(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print(f'Random forest roc_auc: {roc_auc_score(y_train, pred[:,1])}')
    print('Test set')
    pred = rf.predict_proba(X_test)
    print(f'Random forest roc_auc: {roc_auc_score(y_test, pred[:,1])}')

In [16]:
run_randomForest(X_train[selected_feat].fillna(0),
                 X_test[selected_feat].fillna(0),
                y_train, y_test)

Train set
Random forest roc_auc: 0.558023786308931
Test set
Random forest roc_auc: 0.5285220149415576


In [17]:
data = pd.read_csv("../Large_data/feature/house-prices/train.csv",) 
data.shape

(1460, 81)

In [18]:
#feature selection should be done after data preprocessing
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape
#[col for col in data.columns if data[col].isnull().sum() > 0] #presence of null value

(1460, 38)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=["SalePrice"], axis =1),
    data["SalePrice"],
    test_size = 0.3,
    random_state = 0
)
del data
X_train.shape, X_test.shape


((1022, 37), (438, 37))

In [20]:
corr_features = correlation(X_train, 0.8)
print("Correlated Features: ", len(set(corr_features)))

Correlated Features:  3


In [21]:
X_train.drop(labels=corr_features, axis =1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 34), (438, 34))

In [25]:
efs1 = EFS(RandomForestRegressor(n_jobs=4),
           min_features=1,
           max_features=4,
           print_progress=True,
           scoring='r2',
           cv=2)

efs1 = efs1.fit(np.array(X_train[X_train.columns[0:5].fillna(0)]), y_train)

/home/vicky/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

  FitFailedWarning)
/home/vicky/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

  FitFailedWarning)
/home/vicky/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

  FitFailedWarning)
/home/vicky/a

In [26]:
selected_feat = X_train.columns[list(efs1.best_idx_)]
selected_feat

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual'], dtype='object')

In [ ]:
rrr4